[Competitive Data Science Kaggle Competition](https://www.kaggle.com/c/competitive-data-science-final-project)

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics

In [3]:
PATH = 'data/'
!ls {PATH}

item_categories.csv      sales_train.csv.gz       test.csv
items.csv                sample_submission.csv.gz test.csv.gz
sales_train.csv          shops.csv


In [4]:
df_raw = pd.read_csv(f'{PATH}sales_train.csv', low_memory=False, 
                     parse_dates=["date"])

In [6]:
df_raw.sort_values(by='date')

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
109593,2013-01-01,0,46,18616,349.0,1.0
85162,2013-01-01,0,54,11854,199.0,1.0
18128,2013-01-01,0,28,4906,1799.0,1.0
112216,2013-01-01,0,42,2931,99.0,1.0
85141,2013-01-01,0,54,11604,349.0,1.0
47143,2013-01-01,0,15,3686,899.0,1.0
85130,2013-01-01,0,54,11576,149.0,1.0
85129,2013-01-01,0,54,11573,148.0,1.0
85124,2013-01-01,0,54,11562,299.0,1.0
85115,2013-01-01,0,54,11822,349.0,1.0


In [8]:
add_datepart(df_raw, 'date')

Store the df to a file.

In [9]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/data-raw')

In [11]:
df_raw

,date_block_num,shop_id,item_id,item_price,item_cnt_day,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,0,59,22154,999.00,1.0,2013,2,5,1,4,32,False,True,False,False,False,False,1359676800
1,0,25,2552,899.00,1.0,2013,3,9,1,4,60,False,True,False,False,False,False,1362096000
2,0,25,2552,899.00,-1.0,2013,5,18,1,2,121,False,True,False,False,False,False,1367366400
3,0,25,2554,1709.05,1.0,2013,6,22,1,5,152,False,True,False,False,False,False,1370044800
4,0,25,2555,1099.00,1.0,2013,1,3,15,1,15,False,False,False,False,False,False,1358208000
5,0,25,2564,349.00,1.0,2013,10,40,1,1,274,False,True,False,True,False,False,1380585600
6,0,25,2565,549.00,1.0,2013,2,5,1,4,32,False,True,False,False,False,False,1359676800
7,0,25,2572,239.00,1.0,2013,4,14,1,0,91,False,True,False,True,False,False,1364774400
8,0,25,2572,299.00,1.0,2013,11,44,1,4,305,False,True,False,False,False,False,1383264000
9,0,25,2573,299.00,3.0,2013,3,9,1,4,60,False,True,False,False,False,False,1362096000


In [12]:
last_month_df = (df_raw.Month == 10) & (df_raw.Year == 2015)

In [13]:
len(last_month_df)

2935849

In [14]:
sales_previous_month_df = df_raw[last_month_df].groupby(["shop_id", "item_id"], as_index=False).item_cnt_day.sum().rename(columns={"item_cnt_day": "item_cnt_month"})

In [15]:
sales_previous_month_df.tail()

,shop_id,item_id,item_cnt_month
34096,59,22088,2.0
34097,59,22091,1.0
34098,59,22100,1.0
34099,59,22102,1.0
34100,59,22118,1.0


In [16]:
sales_previous_month_df["item_cnt_month"] = sales_previous_month_df["item_cnt_month"].fillna(0)

In [21]:
len(sales_previous_month_df)

34101

In [19]:
def split_vals(a,n): return a[:n], a[n:]

In [20]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [24]:
df, y, nas = proc_df(sales_previous_month_df, 'item_cnt_month')

n_valid = 10000
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

In [43]:
df_trn, y_trn, nas = proc_df(sales_previous_month_df, 'item_cnt_month')

m = RandomForestRegressor(n_jobs=-1)
m.fit(sales_previous_month_df.drop('item_cnt_month', axis=1), sales_previous_month_df.item_cnt_month)
print_score(m)

[4.573125884284436, 1.4338556412693713, 0.91833932995750778, 0.8868877554911756]


### Test csv

In [35]:
test_df = pd.read_csv(f'{PATH}test.csv', low_memory=False)

In [46]:
submission_df = test_df.merge(sales_previous_month_df, on=["shop_id", "item_id"], how="left")[["ID", "item_cnt_month"]]
submission_df['item_cnt_month'] = submission_df['item_cnt_month'].fillna(0).clip(0,20)

In [47]:
submission_df.to_csv('./data/submission.csv', index=False)